In [1]:
import datetime
import tweepy
from tweepy import OAuthHandler
import json
import pandas as pd
import csv
import re
import string
import os
import time

# I've put my API keys in a .py file called API_keys.py
from my_api_keys import api_key, api_key_secret, access_token, access_token_secret

In [2]:
datetime.date.today()

datetime.date(2021, 10, 30)

In [3]:
# Authenticate the Tweepy API
auth = tweepy.OAuthHandler(api_key,api_key_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

In [ ]:
# Note: the search API only goes back 7 days
date_end = '2021-10-31'
search_words = "#cop26 OR #cop26Glasgow -filter:retweets"
count=200

# Notice the differences between searching tweets and users. 
tweets = tweepy.Cursor(api.search_tweets,
                   # tweet_mode is defaulted to short, which only holds the first 140 characters of a Tweet.
                   tweet_mode='extended',
                   q=search_words,
                   until=date_end,
                   result_type='recent',
                   lang='en'
                                        ).items(count)
#     if idx == 50:
#         break
    
#     There's all sort of information you can get from Tweets
#     Find more tweet objects here: https://developer.twitter.com/en/docs/twitter-api/v1/data-dictionary/overview/tweet-object
#     print(item.user.id)
#     print(item.user.screen_name)
#     print(item.user.location)
#     print(item.created_at)
#     print(item.full_text)
#     print('-'*40)
    


In [ ]:
tweets_list = []
for tweet in tweets:
    # Filtering by date
    if  datetime.date.today() == tweet.created_at.date():  
        tweets_list.append([
        item.user.id,
        item.user.screen_name,
        item.user.description,
        item.user.location,
        item.user.followers_count,
        item.user.friends_count,    
        item.created_at,
        item.full_text])

In [8]:
def scrapetweets(search_words, numtweets, numruns):
    
    # Define a for-loop to generate tweets at regular intervals
    # We cannot make large API call in one go. Hence, let's try numruns times
    
    # Define a pandas dataframe to store the date:
    db_tweets = pd.DataFrame(columns = ['user_id','screen_name','description','location','friends_count',
           'followers_count','totaltweets','date_created','retweetcount','full_text','hashtags'])
    
    program_start = time.time()
    for i in range(0, numruns):
        # We will time how long it takes to scrape tweets for each run:
        start_run = time.time()
        
        # Collect tweets using the Cursor object
        # .Cursor() returns an object that you can iterate or loop over to access the data collected.
        # Each item in the iterator has various attributes that you can access to get information about each tweet
        tweets = tweepy.Cursor(api.search_tweets, 
                               q=search_words, 
                               lang="en", 
                               tweet_mode='extended'
                              ).items(numtweets)
        
        # Store these tweets into a python list
        tweet_list = [tweet for tweet in tweets]
        
        # Begin scraping the tweets individually:
        noTweets = 0
    
        for tweet in tweet_list:
            userid = tweet.user.id
            username = tweet.user.screen_name
            description = tweet.user.description
            location = tweet.user.location
            following = tweet.user.friends_count
            followers = tweet.user.followers_count
            totaltweets = tweet.user.statuses_count
            date_created = tweet.created_at
            retweetcount = tweet.retweet_count
            full_text = tweet.full_text
            hashtags = tweet.entities['hashtags']

            # Add the 11 variables to the empty list - ith_tweet:
            ith_tweet = [userid, username, description, location, following, followers, totaltweets,
                         date_created, retweetcount, full_text, hashtags]

            # Append to dataframe - db_tweets
            db_tweets.loc[len(db_tweets)] = ith_tweet

            # increase counter - noTweets  
            noTweets += 1

#             except TweepError as err: 
#                 if err.code == 103: #if we get a rate limit error, go to sleep
#                     print('sleeping, 900 seconds')
#                     time.sleep(900)            
            
            # Run ended:
            end_run = time.time()
            duration_run = round((end_run-start_run)/60, 2)

            print('no. of tweets scraped for run {} is {}'.format(i + 1, noTweets))
            print('time take for {} run to complete is {} mins'.format(i+1, duration_run))

    # Once all runs have completed, save them to a single csv file:
    
    # Obtain timestamp in a readable format
    to_csv_timestamp = datetime.date.today().strftime('%Y%m%d_%H%M%S')
    
    # Define working path and filename
    path = os.getcwd()
    filename = path + '/data/' + to_csv_timestamp + '_cop_tweets.csv'
    
    # Store dataframe in csv with creation date timestamp
    db_tweets.to_csv(filename, index = False)
    
    program_end = time.time()
    print('Scraping has completed!')
    print('Total time taken to scrape is {} minutes.'.format(round(program_end - program_start)/60, 2))

In [9]:
search_words = "#cop26 OR #cop26Glasgow -filter:retweets"
numtweets=100
numruns=1

scrapetweets(search_words, numtweets, numruns)

no. of tweets scraped for run 1 is 1
time take for 1 run to complete is 0.04 mins
no. of tweets scraped for run 1 is 2
time take for 1 run to complete is 0.04 mins
no. of tweets scraped for run 1 is 3
time take for 1 run to complete is 0.04 mins
no. of tweets scraped for run 1 is 4
time take for 1 run to complete is 0.04 mins
no. of tweets scraped for run 1 is 5
time take for 1 run to complete is 0.04 mins
no. of tweets scraped for run 1 is 6
time take for 1 run to complete is 0.04 mins
no. of tweets scraped for run 1 is 7
time take for 1 run to complete is 0.04 mins
no. of tweets scraped for run 1 is 8
time take for 1 run to complete is 0.04 mins
no. of tweets scraped for run 1 is 9
time take for 1 run to complete is 0.04 mins
no. of tweets scraped for run 1 is 10
time take for 1 run to complete is 0.04 mins
no. of tweets scraped for run 1 is 11
time take for 1 run to complete is 0.04 mins
no. of tweets scraped for run 1 is 12
time take for 1 run to complete is 0.04 mins
no. of tweets

FileNotFoundError: [Errno 2] No such file or directory: '/Users/natebender/Desktop/Repo/text-mining/datashare_AA_COP_tweets/data/20211030_000000_cop_tweets.csv'

In [ ]:
headers = ['user_id','screen_name','description','location','followers_count',
           'friends_count','date_created','full_text']


# Descriptions with emoji or non-Roman letters can cause trouble. Encoding your .txt file in utf-8 will help
with open(f'test_cop_tweets.txt','w', encoding='utf-8') as out_file:
    out_file.write('\t'.join(headers) + '\n')

    for tweet in tweets_list:

        # For accounts set to private, we won't be able to get the description unless we follow them
        # Putting in a try/except statement, we can get around this issue.
        user_id = str(user.id).replace('\t',' ').replace('\n',' ')
        screen_name = str(user.screen_name).replace('\t',' ').replace('\n',' ')
        description = str(user.description).replace('\t',' ').replace('\n',' ')
        location = str(user.location).replace('\t',' ').replace('\n',' ')        
        followers_count = str(user.followers_count).replace('\t',' ').replace('\n',' ')        
        friends_count = str(user.friends_count).replace('\t',' ').replace('\n',' ')
        date_created = str(item.created_at).replace('\t',' ').replace('\n',' ')        
        full_text = str(item.full_text).replace('\t',' ').replace('\n',' ')        
        
        outline = [user_id, screen_name, description, location, followers_count, friends_count,
                  date_created, full_text]

        out_file.write('\t'.join([str(item) for item in outline]) + '\n')
